In [4]:
import os
import re
import glob
import datetime
import warnings

import numpy as np
import pandas as pd
import xarray as xr

In [9]:
%%time
## This section caches all MODIS MCD43A4 file names in the format listed below, where '?' is a wildcard


reflectance_file_cache =  sorted(glob.glob(
    '/g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/'
    '????.??.??/MCD43A4.A???????.h??v??.006.*.hdf'
))
print(len(reflectance_file_cache))

505518
CPU times: user 6.76 s, sys: 6.36 s, total: 13.1 s
Wall time: 7min 53s


In [13]:
## Save .txt file with all MODIS file names
with open('/g/data/oe9/modis_tile_file_list.txt', 'w') as f:
    f.writelines(l + '\n' for l in reflectance_file_cache
# this .txt file has been saved in git, Rong branch, only run this cell for the first time

In [ ]:
## Rename the bands to something useful
modis_band_map = {
    'Nadir_Reflectance_Band1': 'red_630_690',
    'Nadir_Reflectance_Band2': 'nir1_780_900',
    'Nadir_Reflectance_Band3': 'blue_450_520',
    'Nadir_Reflectance_Band4': 'green_530_610',
    'Nadir_Reflectance_Band5': 'nir2_1230_1250',
    'Nadir_Reflectance_Band6': 'swir1_1550_1750',
    'Nadir_Reflectance_Band7': 'swir2_2090_2350',
}

## define function 'get_reflectance' whose input is tile ID 'h__v__'
## This function searches the cached list for filenames which contain tile 'h__v__
## outputs data as a concatenated dataset with dimensions, time, x ,y 
def get_reflectance(tile):
    assert re.match(r'h\d\dv\d\d', tile), 'tile must be string "h__v__"'
    files = [f for f in reflectance_file_cache if tile in os.path.basename(f)]
    pattern = re.compile(r'MCD43A4.A\d{4}(?P<day>\d{3}).h\d\dv\d\d.006.\d+.hdf')
    dates, parts = [], []
    for f in files:
        try:
            parts.append(xr.open_dataset(f, chunks=2400,autoclose=True))
            day, = pattern.match(os.path.basename(f)).groups()
            dates.append(datetime.date(int(year), 1, 1) +
                         datetime.timedelta(days=int(day) - 1))
        except:
            warnings.warn('Could not read from ' + f)

    dates = pd.to_datetime(dates)
    dates.name = 'time'

    ds = xr.concat(parts, dates)
    out = xr.Dataset()
    for i in map(str, range(1, 8)):
        key = 'Nadir_Reflectance_Band' + i
        data_ok = ds['BRDF_Albedo_Band_Mandatory_Quality_Band' + i] == 0
        out[modis_band_map[key]] = ds[key].where(data_ok).astype('f4')

    out.rename({'YDim:MOD_Grid_BRDF': 'y',
                'XDim:MOD_Grid_BRDF': 'x'}, inplace=True)
    out.time.encoding.update(dict(
        units='days since 1900-01-01', calendar='gregorian', dtype='i4'))
    return out

In [ ]:
%%time
ds = get_reflectance('h29v12')
ds

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.02.26/MCD43A4.A2000057.h29v12.006.2016101183811.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.03.01/MCD43A4.A2000061.h29v12.006.2016101213834.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.03.05/MCD43A4.A2000065.h29v12.006.2016101230029.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.03.09/MCD43A4.A2000069.h29v12.006.2016102020642.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.07.19/MCD43A4.A2000201.h29v12.006.2016104081238.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.07.23/MCD43A4.A2000205.h29v12.006.2016104094406.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.07.27/MCD43A4.A2000209.h29v12.006.2016104110613.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.07.31/MCD43A4.A2000213.h29v12.006.2016104122228.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.12.10/MCD43A4.A2000345.h29v12.006.2016110214805.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.12.14/MCD43A4.A2000349.h29v12.006.2016110230841.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.12.18/MCD43A4.A2000353.h29v12.006.2016111003050.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2000.12.22/MCD43A4.A2000357.h29v12.006.2016111014326.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.05.05/MCD43A4.A2001125.h29v12.006.2016114030142.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.05.09/MCD43A4.A2001129.h29v12.006.2016114040526.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.05.13/MCD43A4.A2001133.h29v12.006.2016114045201.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.05.17/MCD43A4.A2001137.h29v12.006.2016114054405.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.09.26/MCD43A4.A2001269.h29v12.006.2016118214413.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.09.30/MCD43A4.A2001273.h29v12.006.2016118224051.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.10.04/MCD43A4.A2001277.h29v12.006.2016118233257.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2001.10.08/MCD43A4.A2001281.h29v12.006.2016119002806.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.02.18/MCD43A4.A2002049.h29v12.006.2016124185539.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.02.22/MCD43A4.A2002053.h29v12.006.2016124194103.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.02.26/MCD43A4.A2002057.h29v12.006.2016124205222.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.03.02/MCD43A4.A2002061.h29v12.006.2016124220425.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.07.16/MCD43A4.A2002197.h29v12.006.2016128205136.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.07.20/MCD43A4.A2002201.h29v12.006.2016128220116.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.07.24/MCD43A4.A2002205.h29v12.006.2016131133228.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Could not read from /g/data/u39/public/data/modis/lpdaac-tiles-c6/MCD43A4.006/2002.07.28/MCD43A4.A2002209.h29v12.006.2016131150101.hdf
/g/data/oe9/software/anaconda/lib/python3.6/site-packages/ipykernel_laun

[]